In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
code_data=pd.read_csv('data/code_data.csv')
code=[]
for i in range(len(code_data)):
    code.append(code_data[['code']].iloc[i][0])

In [3]:
original_data=pd.read_csv('data/data.csv')

df_close = pd.DataFrame()
df_date = []
for i in range(len(code)): 
    data = original_data
    data = original_data[original_data['code']==code[i]].reset_index().sort_values(by=['date'])
    data = data[['date','close']]
    data = data.set_index('date').loc['20170101':'20200223'] # 3의 배수 맞추기위해 날짜 조정
    df_close = pd.concat([df_close, data], axis=1)

df_close.columns = ['a{}' .format(i) for i in code]
# 현재 코스피 200 종목중 2년이상 존재 하지 않으면 삭제
df_close = df_close.dropna(axis=1) 

# 인덱스 변경해주기
# 원래 설정대로 날짜를 이용하면 그래프가 제대로 표현이 되지 않음.
df_date.append(df_close.index)
date=[]
for i in df_date[0]:
    date.append('{}-{}-{}'.format(str(i)[0:4],str(i)[4:6],str(i)[6:]))
df_close.index =date

In [4]:
dic={'aa':'c01','ab':'c02','ac':'c03','ad':'c04','ae':'c05',
     'ba':'c06','bb':'c07','bc':'c08','bd':'c09','be':'c10',
     'ca':'c11','cb':'c12','cc':'c13','cd':'c14','ce':'c15',
     'da':'c16','db':'c17','dc':'c18','dd':'c19','de':'c20',
     'ea':'c21','eb':'c22','ec':'c23','ed':'c24','ee':'c25'}



# <범위 정하기>
# return = ((종목[열][1] / 종목[열][0]) -1)*100
# 1. -0.3 < return < 0.3
# 2. 0.3 < return < 1.0
# 3. -1.0 < return < -0.3
# 4. return >1.0
# 5. return <-1.0

In [5]:
final_class=[]

for j in [i for i in df_close.columns]:
    stock = pd.DataFrame(np.array(df_close[j]).reshape(-1,3)).T
    stock.columns = ['b{}' .format(i) for i in range(len(stock.columns))]
    
    percentage=[]
    for k in stock.columns:
        for l in range(2):
            if ((stock[k][l+1]/stock[k][l])-1)*100 < -1.0:
                percentage.append('a')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= -1.0  and ((stock[k][1]/stock[k][l])-1)*100 < -0.3:
                percentage.append('b')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= -0.3 and ((stock[k][1]/stock[k][l])-1)*100 < 0.3:
                percentage.append('c')
            elif ((stock[k][l+1]/stock[k][l])-1)*100 >= 0.3 and ((stock[k][1]/stock[k][l])-1)*100 < 1.0:
                percentage.append('d')
            else :
                percentage.append('e')
    
    class_per = pd.DataFrame(np.array(percentage).reshape(-1,2)).T
    class_per.columns = ['b{}' .format(i) for i in range(len(stock.columns))]
    class_per.index = ['return_1','return_2']
    stock = pd.concat([stock, class_per], axis=0)
    
    class_=[]
    for m in range(stock.shape[1]):
        a=(stock.iloc[3,m]+stock.iloc[4,m])
        class_.append(dic[a])
    
    stock.loc[6]=class_
    stock.index = ['first_price','second_price','third_price','first_return','second_price','class']
    final_class.append(stock.loc['class',].tolist())

In [6]:
final = pd.DataFrame(np.array(final_class).reshape(188,-1))
final.index = [i for i in df_close.columns]
final = final.T
#final.to_csv('data/class.csv')

In [7]:
final.head()

,a5930,a660,a35420,a51910,a5380,a68270,a6400,a12330,a28260,a51900,...,a25860,a2240,a2960,a25540,a1520,a96760,a6840,a3620,a3920,a79430
0,c25,c21,c03,c23,c23,c05,c21,c23,c23,c23,...,c13,c03,c15,c01,c08,c18,c01,c13,c03,c13
1,c23,c23,c21,c10,c05,c01,c18,c03,c20,c23,...,c21,c16,c15,c21,c06,c01,c16,c01,c06,c21
2,c23,c21,c18,c23,c05,c11,c13,c08,c08,c08,...,c08,c23,c18,c23,c10,c06,c15,c03,c08,c08
3,c03,c03,c18,c03,c23,c06,c03,c13,c06,c23,...,c08,c03,c08,c16,c01,c01,c20,c13,c03,c03
4,c25,c25,c21,c06,c03,c08,c03,c03,c16,c23,...,c03,c06,c03,c03,c13,c03,c13,c03,c03,c23
